In [38]:
from keras import Sequential
import keras
import os
from keras.layers import Flatten, Dense, SimpleRNN, LSTM, BatchNormalization, Conv1D, Dropout
import tensorflow as tf
import numpy as np
# import Weather_Data as WD
from time import time

# Убираем предупреждения
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
tf.get_logger().setLevel(logging.ERROR)



def what_device_use(device="cpu"):
    if device.lower() == "gpu":
        # Работаем с GPU
        tf.config.set_visible_devices(tf.config.list_physical_devices("GPU"), "GPU")

    if device.lower() == "cpu":
        # Работаем с CPU
        tf.config.set_visible_devices([], "GPU")


def load_data(name_db="moscow", len_test_data=0):
    """Загружаем данные"""
    global train_data, train_data_answer, test_data, test_data_answer

    # В WD.get_moscow_data     477_603 записей      (все данные идут с шагом в 1 часа)
    # В WD.get_fresh_data        1_455 записей      (данные за последние 60 дней, идут с шагом в 1 час)

    if name_db == "moscow":
        DATA_out = WD.get_moscow_data()
    elif name_db == "fresh":
        DATA_out = WD.get_fresh_data()
    print(">>> Dataset loaded\n")


    """DATA_in == Данные погоды, начиная с 1ого дня (принимает)
       DATA_out == Данные погоды, начиная с 2ого дня (должен предсказать)"""

    # Создаём смещени назад во времени
    DATA_in = DATA_out[:-1]
    DATA_out = DATA_out[1:]

    # Преобразуем данные
    DATA_out = np.array(DATA_out).reshape((len(DATA_out), 1, 8))
    DATA_in = np.array(DATA_in).reshape((len(DATA_out), 1, 8))

    DATA_out = WD.normalize(DATA_out - DATA_in) # Остаточное обучение + нормализуем от -1 до 1 (а не от -0.01 до 0.01)
    DATA_out = DATA_out[:, :, 3:]               # ИИшке не надо предсказывать время

    # Разделяем часть для обучения и для тестирования
    train_data = DATA_in[:-len_test_data] if len_test_data > 0 else DATA_in
    train_data_answer = DATA_out[:-len_test_data] if len_test_data > 0 else DATA_out

    if len_test_data > 0:
        test_data = DATA_in[-len_test_data:]
        test_data_answer = DATA_out[-len_test_data:, 0, :]
    else:
        test_data, test_data_answer = [], []


def create_ai(num_layers_conv=3, num_ai_layers=5, num_neurons=32, print_summary=True):
    """Создаём ИИшки"""
    global ai
    # Суть в том, чтобы расперелить задачи по предсказыванию между разными нейронками
    # Т.к. одна нейросеть очень плохо предскаывает одновременно все факторы

    # У всех нейронок одна архитектура и один вход
    input_layer = keras.Input((1, 8))


    class Architecture:
        def get_ai(self):
            num_conv_neurons = 8
            # Добавляем нормализаци, т.к. некоторый значения больше склонны бать с одной стороны (если взять среднее)
            # (например температура чаще положительная, чем отрицательная (в москве))
            list_layers = []

            # Добавляем Conv1D
            for _ in range(num_layers_conv):
                list_layers.append(Conv1D(num_conv_neurons, 8, padding="same"))
                num_conv_neurons *= 2

            # Добавляем остальные слои
            for i in range(num_ai_layers):
                if i % 2 == 0:
                    list_layers.append(LSTM(num_neurons, return_sequences=True, unroll=True))
                else:
                    list_layers.append(Dense(num_neurons, activation="relu"))

            return Sequential(list_layers)(input_layer)

    # Создаём 5 полностью независимые нейронки
    temperature = Dense(1, activation="tanh", name="temp")(Architecture().get_ai())
    pressure = Dense(1, activation="tanh", name="press")(Architecture().get_ai())
    humidity = Dense(1, activation="tanh", name="humid")(Architecture().get_ai())
    cloud = Dense(1, activation="tanh", name="cloud")(Architecture().get_ai())
    rain = Dense(1, activation="tanh", name="rain")(Architecture().get_ai())


    ai = keras.Model(input_layer, [temperature, pressure, humidity, cloud, rain], name="Weather_Predictor")
    ai.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mean_squared_error",
               loss_weights={"temp": 100_000, "press": 10_000, "humid": 10_000, "cloud": 10_000, "rain": 100_000},)
               # Отдаём приоритет температуре и осадкам, и увеличиваем ошибки (иначе они будут ≈0)

    if print_summary:
        ai.summary(); print()


def ai_name(name):
    """Сохранения / Загрузки"""
    global save_path, SAVE_NAME

    save_path = lambda ai_name: f"Saves Weather Prophet/{ai_name}"
    SAVE_NAME = lambda num: f"{name}~{num}"


def load_ai(loading_with_learning_cycle=-1, print_summary=False):
    """ЗАГРУЖАЕМСЯ"""
    global ai

    # Вычисляем номер последнего сохранения с текущем именем
    if loading_with_learning_cycle == -1:
        loading_with_learning_cycle = int(sorted([save_name if SAVE_NAME(0)[:-2] in save_name
                    else None for save_name in os.listdir("Saves Weather Prophet")])[-1].split("~")[-1])

    print(f">>> Loading the {SAVE_NAME(loading_with_learning_cycle)}", end="\t\t")
    ai = tf.keras.models.load_model(save_path(SAVE_NAME(loading_with_learning_cycle)))
    print("Done\n")

    if print_summary:
        ai.summary(); print()


def train_ai(start_on=-1, finish_on=99, # Начинаем с номера последнего сохранения до finish_on
             save_every_learning_cycle=True,    # Сохранять ли каждую ИИшку
             epochs=3, batch_size=100,  verbose=2, # Параметры fit()
             print_ai_answers=True, len_prints_ai_answers=100, # Выводить и сравнивать данные, или нет
             print_weather_predict=True, len_predict_days=3, # Выводить ли  прогноз погоды
             use_callbacks=False, callbacks_min_delta=10, callbacks_patience=3, # Параметры callbacks
             shift_dataset_every_cycle=True, start_with_dataset_offset=0, # Смещаем данные на 1 час каждый цикл
                     # (т.е. после первого смещения ИИшка должна предсказывать на 2 часа вперёд, потом на 3...)
             ):
    """Обучение"""
    global train_data, train_data_answer, test_data, test_data_answer
    num_dataset_offset = 1

    # Сдвигаемм наборы данных
    if start_with_dataset_offset > 0:
        num_dataset_offset += start_with_dataset_offset
        train_data = train_data[: -start_with_dataset_offset]
        train_data_answer = train_data_answer[start_with_dataset_offset:]
        if len(train_data) > 0:
            test_data = test_data[: -start_with_dataset_offset]
            test_data_answer = test_data_answer[start_with_dataset_offset:]


    callbacks = [keras.callbacks.EarlyStopping(monitor="loss",
                min_delta=callbacks_min_delta, patience=callbacks_patience, verbose=False)] \
        if use_callbacks else None

    # Продолжаем с последнего сохранения если start_on == -1 (или создаём новое)
    if start_on == -1:
        try:
            start_on = int(sorted([save_name if SAVE_NAME(0)[:-2] in save_name else None
                            for save_name in os.listdir("Saves Weather Prophet")])[-1].split("~")[-1]) +1
        except:
            start_on = 0

    # Циклы обучения
    for learning_cycle in range(start_on, finish_on):
        print(f">>> Learning the {SAVE_NAME(learning_cycle)}\t\t\tПредсказывает на: {num_dataset_offset} ч вперёд")
        ai.fit(train_data, train_data_answer,
               epochs=epochs, batch_size=batch_size,
               verbose=verbose, shuffle=False, callbacks=callbacks)
        print()


        # Сохраняем
        if save_every_learning_cycle:
            print(f">>> Saving the {SAVE_NAME(learning_cycle)}  (Ignore the WARNING)", end="\t\t")
            ai.save(save_path(SAVE_NAME(learning_cycle)))
            print("Done\n")


        # Выводим данные и сравниваем
        if print_ai_answers:
            # Используем train_data если test_data нет
            WD.print_ai_answers(ai, test_data if len(test_data)>0 else train_data, len_prints_ai_answers)
        if print_weather_predict:
            WD.print_weather_predict(ai, len_predict_days)


        # Создаём смещение данных на 1 час
        if shift_dataset_every_cycle:
            num_dataset_offset += 1
            train_data = train_data[: -1]
            train_data_answer = train_data_answer[1:]
            if len(train_data) > 0:
                test_data = test_data[: -1]
                test_data_answer = test_data_answer[1:]


In [39]:
what_device_use("gpu")
ai_name("AI_v4.2")
load_data("moscow", len_test_data=0)

>>> Dataset loaded



In [40]:
# create_ai(5, 5, 128, print_summary=True)
load_ai(-1, print_summary=True)

>>> Loading the AI_v4.2~10		Done

Model: "Weather_Predictor"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 8)]       0           []                               
                                                                                                  
 sequential (Sequential)        (None, 1, 128)       515576      ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 1, 128)       515576      ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 1, 128)       515576      ['input_1[0][0]']                
                                                

In [40]:
def print_weather_predict(ai, len_predict_days=3):
    print(f"Prediction for the next {len_predict_days} days:\t\t\t",
          f"(Temperature, Pressure, Humidity, Cloud, Raininess)")

    # Самое последнее - самое свежее
    # Также сразу подаём только amount_available_context данных для прогноза
    fresh_data = np.reshape(np.array(get_fresh_data(len_predict_days * 24)),
                            (len_predict_days * 24, 8))[::-1]
    times = fresh_data[:, :3].tolist()
    predicts_history = fresh_data[:, 3:].tolist()


    for _ in range(0, len_predict_days*24):
        # Делаем прогноз по всей истории, а потом отбираем один прогноз, относящееся к последней записи
        # preds_on_preds = ai.predict(np.array( [[t + p] for t, p in zip(times, predicts_history)] ), verbose=False)
        # ai_ans = np.reshape(np.array(preds_on_preds)[:, -1], (5))
        preds_on_preds = ai.predict(np.array([[times[-1] + predicts_history[-1]]]), verbose=False)
        ai_ans = np.reshape(np.array(preds_on_preds), (5))
        ai_ans = normalize(ai_ans, True)

        # predicts_history.append( (ai_ans + np.array(predicts_history[-1])).tolist() )
        predicts_history.append( (ai_ans + np.array(predicts_history)).tolist() )
        # predicts_history = predicts_history[1:]

        # Обновляем время
        time = times[-1]
        time[0] += 1/12                          # Увеличиваем часы
        time[1] += 1/15.5 if time[0] >1 else 0   # Увеличиваем день
        time[2] += 1/6    if time[1] >1 else 0   # Увеличиваем месяц

        # Следим, чтобы зачения не выходили за границы
        time = [-1 if i>1 else i for i in time]

        times.append(time)


        # Выводим прогноз
        time_for_human = [norm_hours(time[0], True),
                          norm_day(time[1], True),
                          norm_month(time[2], True)]

        pred_weather_for_human = conv_ai_ans_for_human(predicts_history[-1])
        print(f"{'{:02}'.format(time_for_human[1])}.{'{:02}'.format(time_for_human[2])}",
              f"{'{:02}'.format(time_for_human[0])}:00:\t",
              pred_weather_for_human[0], "℃\t",
              pred_weather_for_human[1], "mmHg\t",
              pred_weather_for_human[2], "%\t",
              pred_weather_for_human[3], "%\t",
              conv_rain_to_words(pred_weather_for_human[4]),
              )
    print("\n")

In [42]:
print_weather_predict(ai, 3)

Prediction for the next 3 days:			 (Temperature, Pressure, Humidity, Cloud, Raininess)
[[0.7739083398558421, -0.28673029138880246, 0.16, 1.0, 0.0], [0.7739083398558421, -0.29129922664175373, 0.19333333333333336, 1.0, 0.0], [0.7739083398558421, -0.29586816189470494, 0.22666666666666668, 1.0, 0.0], [0.7739083398558421, -0.3004370971476562, 0.26, 1.0, 0.0], [0.7640311823578405, -0.30496503988077, 0.34, 1.0, 0.16666666666666669], [0.7541540248598388, -0.3094929826138838, 0.42000000000000004, 1.0, 0.33333333333333337], [0.7442768673618373, -0.3140209253469976, 0.5, 1.0, 0.5], [0.7427789286658929, -0.3229480567705778, 0.48, 1.0, 0.5], [0.7412809899699486, -0.331875188194158, 0.46, 1.0, 0.5], [0.7397830512740042, -0.3408023196177382, 0.44, 1.0, 0.5], [0.7191160677982793, -0.331875188194158, 0.52, 1.0, 0.41666666666666663], [0.6984490843225544, -0.3229480567705778, 0.6000000000000001, 1.0, 0.3333333333333333], [0.6777821008468294, -0.3140209253469976, 0.68, 1.0, 0.25], [0.6490278657084918, -0.

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
# train_ai(epochs=1, batch_size=500, verbose=1, start_with_dataset_offset=0,
#          shift_dataset_every_cycle=False)